In [ ]:
import pandas as pd
import numpy as np
import jsonlines
import gensim
import nltk
import re

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
nltk.download('punkt')

## Data Exploration

In [ ]:
resume_df = pd.read_csv('gs://path-to-data')

In [ ]:
resume_df.head()

In [ ]:
resume_df.rename(columns = {'ID':'candidate_id', 'Category': 'job_role', 'Resume_str': 'raw_resume_txt'}, inplace = True)
resume_df.drop(["Resume_html"], axis=1, inplace=True)

In [ ]:
resume_df.head()

In [ ]:
resume_df['job_role'].value_counts().plot(kind="bar")

## Feature Engineering/ Data Cleansing

In [ ]:
def clean_text(raw_text):
    text = raw_text.lower()
    text = re.sub('[\n\xa0]', '', text)
    text = re.sub(' +', ' ', text)
    text = re.sub(' , ', ' ', text)
    text = re.sub('[,\/|]', '', text)
    return text

In [ ]:
# clean_text(resume_df[resume_df.candidate_id == 36856210]["raw_resume_txt"].values[0])

In [ ]:
resume_df["clean_resume"] = resume_df.apply(lambda row: clean_text(row["raw_resume_txt"]), axis=1)

In [ ]:
resume_df["candidate_id"] = resume_df["candidate_id"].astype(int)

In [ ]:
resume_df["tagged_doc"] = resume_df.apply(
    lambda row: TaggedDocument(
        word_tokenize(row["clean_resume"]), [row['candidate_id']]), 
    axis=1)

In [ ]:
resume_df.head()

## Training Doc2Vec NLP model

In [ ]:
## Train doc2vec model
find_candidates = Doc2Vec(list(resume_df["tagged_doc"]), vector_size=25, window=2, min_count=1, workers=4, epochs = 100)
# Save trained doc2vec model
find_candidates.save("match_candidate.model")
## Load saved doc2vec model
find_candidates = Doc2Vec.load("match_candidate.model")

## Find top suited candidates

In [ ]:
job_description = ""

In [ ]:
jd = clean_text(job_description)
jd_it = word_tokenize(jd)

In [ ]:
# find_candidates.docvecs.most_similar(positive=[find_candidates.infer_vector(jd_it)],topn=5)